In [12]:
import pandas as pd
import sqlite3
import config

def query(sql: str) -> pd.DataFrame:
    """
    Execute a query and yield the results
    """
    conn = sqlite3.connect(config.DB_PATH)
    return pd.read_sql_query(sql, conn)

In [15]:
data = query("SELECT * FROM ledger_items")
data

,tx_id,tx_date,tx_datetime,amount,currency,description,account,ledger_item_type
0,None,2023-02-18,2023-02-18 00:00:00,-250,EUR,"BANCOMAT,Operazione bancomat",default,expense
1,None,2023-02-17,2023-02-17 00:00:00,-1,EUR,MULTIFUNZIONE CONTACTLESS CHIP 4030 **** **** ...,default,expense
2,None,2023-02-17,2023-02-17 00:00:00,-6,EUR,MULTIFUNZIONE CONTACTLESS CHIP 4030 **** **** ...,default,expense
3,None,2023-02-16,2023-02-16 00:00:00,-6,EUR,MULTIFUNZIONE CONTACTLESS CHIP 4030 **** **** ...,default,expense
4,None,2023-02-16,2023-02-16 00:00:00,-14,EUR,MULTIFUNZIONE CONTACTLESS CHIP 4030 **** **** ...,default,expense
...,...,...,...,...,...,...,...,...
198,None,2022-11-22,2022-11-22 00:00:00,-12,EUR,"SEPA Direct Debit,satispay europe s.a. Addebit...",default,expense
199,None,2022-11-22,2022-11-22 00:00:00,-25,EUR,"SEPA Direct Debit,Salmoiraghi + Vigano Spa by ...",default,expense
200,None,2022-11-22,2022-11-22 00:00:00,-25,EUR,"SEPA Direct Debit,Salmoiraghi + Vigano Spa by ...",default,expense
201,None,2022-11-20,2022-11-20 00:00:00,-35.880000000000002557953848736360669136047363...,EUR,MULTIFUNZIONE CONTACTLESS CHIP 4030 **** **** ...,default,expense
